<a href="https://colab.research.google.com/github/jimxs74/SC/blob/sc62_new-dashBoard/SC62_LembarKoordinasi_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SC62 Script Lembar Koordinasi**

Tool ini berguna untuk mendapatkan gambaran progress overall SC project secara angka. agar bisa diidentifikasi letak work volume terbanyak ada diSystem/Disiplin mana

UI/UX belum menjadi objective dari tool ini, akan dilanjutkan dengan framework yang lain

In [1]:
import pandas as pd
import numpy as np
import gspread

In [2]:
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()

import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [3]:
def read_data(url, sheet='sheet1'):
  gs = gc.open_by_url(url).worksheet(sheet).get_all_values()
  gs = pd.DataFrame.from_records(gs)
  gs.columns = gs.iloc[0]
  gs = gs[1:]
  return gs

def write_data(url, result, sheet):
  write = gc.open_by_url(url)
  write = write.worksheet(sheet)
  write.clear()
  set_with_dataframe(worksheet=write, dataframe=result, include_index=False,include_column_header=True, resize=True)
  return

In [12]:
# gsheet yang akan diolah dan analisis
url_sc18 = 'https://docs.google.com/spreadsheets/d/1wrvBoSNka1f51hk8QuLd_bF7irVzSB6nDpg-Z_XqcG4/edit#gid=532748829'
url_cert = "https://docs.google.com/spreadsheets/d/1xPQanRi1XDmo7FlUyiV6Rt8pro_zjEHLOE0GLROTReQ/edit#gid=1035965456"
url_itr = "https://docs.google.com/spreadsheets/d/1aB57SwMedoMSjBxt5w_EBNYgmMoUACTJC056gEbD9a0/edit#gid=1491426671"
url_cc = "https://docs.google.com/spreadsheets/d/1ZrQjq30E58bFFiuuZ9aqAYy2StB4uKDRb2oDtM3m8wk/edit#gid=1688767959"
url_pc = "https://docs.google.com/spreadsheets/d/1DHwpGa_I0sd42o9zfC5cUMSud9XfKwPtyYm7UHY9xl0/edit#gid=1494113475"
url_pl = "https://docs.google.com/spreadsheets/d/19VIlyxleK14LjZCPls-4F9fjg0Sh46Q8oTk5UawJzhI/edit#gid=622809212"
url_sr = "https://docs.google.com/spreadsheets/d/1X3Zy083zX2spgZWqlw9TfnzUpHh_LR_C3H1jXyEdB4A/edit#gid=253348165"

# Output tampilan
url_sc62 = 'https://docs.google.com/spreadsheets/d/1BRdjDg45HbNN1oeN22UwpaBwTL7fDTqOkMTwBHs0g5g/edit#gid=2092045117'

#dummy Test
url_sc62_dummy = 'https://docs.google.com/spreadsheets/d/1nO9qBfJCS8vH7shuc_BAj9OGNsdjwqXkhBhFvhNdAcI/edit#gid=0'

Buat tabel system owner dan subsystem owner

In [13]:
gs_read = gc.open_by_url(url_sc18).worksheet('System List')
gs_write = gc.open_by_url(url_sc62)

df = gs_read.get_all_values()
df = pd.DataFrame.from_records(df)

# Ambil kolom yg di perlukan saja dari SC18
df = df[[1,2,3,4,24,32]] 
df.columns = ['system','system desc','subsystem','subsystem desc', 'SSHOC or SHOC','system owner']
df['system'].replace('',np.nan, inplace=True)
df['subsystem'].replace('',np.nan, inplace=True)

# pilih row yang diperlukan saja
so = df[6:].drop(['subsystem','subsystem desc'], axis=1)

# drop row yg mengandung blank pada column subsystem
so.dropna(subset=['system'], inplace=True)
so.reset_index(drop=True)

# Generate summary table Sub-System Owner dari SC18
# pilih row yang diperlukan saja
sso = df[6:].drop(['system','system desc'], axis=1)

# drop row yg mengandung blank pada column subsystem
sso.dropna(subset=['subsystem'], inplace=True)
sso.reset_index(drop=True)


,subsystem,subsystem desc,SSHOC or SHOC,system owner
0,009-01,Temporary Power MV (20/11kV) and LV (400V),SSHOC,Yadi
1,009-02,Temporary Water Supply,SSHOC,Fathur Rohman
2,009-03,Temporary Safety and Fire Equipment,SSHOC,Hasan H
3,009-04,Temporary HVAC,,Fathur Rohman
4,009-06,Temporary Buyback Gas,SSHOC,Estu YP
...,...,...,...,...
972,870-23,Fire Area-12 (Substation-3),SSHOC,Noprian
973,870-24,Fire Area-14 (Substation-4),SSHOC,Noprian
974,870-25,Fire Area-15 (Control Building),SSHOC,
975,999-01,,,


In [14]:
so_sso = pd.concat([so, sso], axis=1)

In [20]:
so.columns = ['system/subsystem','desc', 'SSHOC/SHOC','system owner']
sso.columns = ['system/subsystem','desc', 'SSHOC/SHOC','system owner']
so_sso = pd.concat([so, sso])

In [22]:
#write tabel so dan sso tabel ini tidak perlu ditampilkan dalam frontend gsheet
write_data(url_sc62_dummy, so, 'so')
write_data(url_sc62_dummy, sso, 'sso')
write_data(url_sc62_dummy, so_sso, 'so_sso')

#1. Connect ke Data

In [23]:
# link data dari p Supri. di update harian oleh p Supri
cert = read_data(url_cert,'CERT')
itr = read_data(url_itr,'ITR')
cc = read_data(url_cc,'CC')
pc = read_data(url_pc,'PC')
pl = read_data(url_pl,'PL')
sr = read_data(url_sr,'PHASE')

#2. Certificate connect dengan subsystem owner (cert+sso)

In [24]:
cert = cert[['Certificate No','System No','System Description','Sub System No','Sub System Description','Discipline','Status']]


In [25]:
cert.head(3)

,Certificate No,System No,System Description,Sub System No,Sub System Description,Discipline,Status
1,FAN-JTB,,,,,,Outstanding
2,PSSR-JTB,,,,,,Outstanding
3,RHIC-JTB,,,,,,Outstanding


 kelebihan : join system owner lebih specifik pada subsystem, lebih akurat <br>
 kekurangan : ada bug, SHOC/SCCC/RFSU tidak terambil datanya dan teralamatkan ke PIC SO

In [ ]:
#ambil data certificate
cert = cert[['Certificate No','System No','System Description','Sub System No','Sub System Description','Discipline','Status']]
#cert['System No'] = pd.to_numeric(cert['System No'], errors='coerce').astype('Int64')

#ambil data SC61 worksheet sso
sso = read_data(url_sc62,'sso')

# Merge data dengan CSystem Report Certificate Status dengan SC61
sert_sso = pd.merge(cert, sso, left_on ="Sub System No", right_on="subsystem" , how="left") # previous is inner
sert_sso = sert_sso.drop(['System No','System Description','subsystem','subsystem desc'], axis=1)

# Sebaran Outstanding Certificate
sert_sso = sert_sso[sert_sso['Status'].str.contains('Outstanding')]
sso_load= sert_sso["system owner"].value_counts()
#print("Beban System Owner dalam subsystem lebih akurat : ")
#print(sso_load)

In [ ]:
# save to gsheet diworksheet sso
#write_data(url_sc62, sert_sso, sheet='sc_workfront')

In [ ]:
#update ke data system owner sertificate, untuk fungsi vlookup di lembar koordinasi
so = read_data(url_sc62,'so')
sert_so = pd.merge(cert, so, left_on ="System No", right_on="system" , how="left") # previosly inner
sert_so = sert_so.drop(['Sub System No','Sub System Description','system','system desc'], axis=1)
sert_so = sert_so.sort_values(by=['System No'])

write_data(url_sc62, sert_so, sheet='so_wf_sert')

#3. Connect to Summary Report (cert+sso+summary report)

In [ ]:
#sr = read_data(url_sr,'PHASE')

In [ ]:
sr = sr[['System No', 'System Description', 'Sub System No','Sub System Description', 'Discipline', 'ITR Outstanding', 'CC Checksheet Outstanding', 'PC Checksheet Outstanding', 'COMM Checksheet Outstanding']]
#sr = sr.fillna(0)

sr['System No'] = pd.to_numeric(sr['System No'], errors='coerce').astype('Int64')
sr['ITR Outstanding'] = pd.to_numeric(sr['ITR Outstanding'], errors='coerce').astype('Int64')
sr['CC Checksheet Outstanding'] = pd.to_numeric(sr['CC Checksheet Outstanding'], errors='coerce').astype('Int64')
sr['PC Checksheet Outstanding'] = pd.to_numeric(sr['PC Checksheet Outstanding'], errors='coerce').astype('Int64')
sr['COMM Checksheet Outstanding'] = pd.to_numeric(sr['COMM Checksheet Outstanding'], errors='coerce').astype('Int64')

sr = sr.fillna(0)

In [ ]:
# Filter yg masih outstanding saja di bagian CC Checksheet
sr_cc = sr[['Sub System No','Discipline','CC Checksheet Outstanding']]
sr_cc = sr_cc[sr_cc['CC Checksheet Outstanding'] != 0]

# combine text on different column
sr_cc["Discipline"] = sr_cc["Discipline"].astype(str) + ":" + sr_cc["CC Checksheet Outstanding"].astype(str)
sr_cc = sr_cc.drop(columns=['CC Checksheet Outstanding'])
sr_cc = sr_cc.set_index('Sub System No')

# combine value as string ref : https://stackoverflow.com/questions/59914167/putting-all-the-same-value-indices-into-one-row
sr_cc = sr_cc.assign(labels = sr_cc.groupby(level = 'Sub System No').cumcount()).groupby([sr_cc.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)
sr_cc.columns = ['CC Outstanding', 'A','B','C']

sr_cc["CC Outstanding"] = sr_cc["CC Outstanding"].astype(str) + "/" + sr_cc["A"].astype(str) + "/" + sr_cc["B"].astype(str) + "/" + sr_cc["C"].astype(str)
sr_cc = sr_cc.drop(columns=['A','B','C'])

In [ ]:
# Filter yg masih outstanding saja di bagian PC Checksheet
sr_pc = sr[['Sub System No','Discipline','PC Checksheet Outstanding']]
sr_pc = sr_pc[sr_pc['PC Checksheet Outstanding'] != 0]

# combine text on different column
sr_pc["Discipline"] = sr_pc["Discipline"].astype(str) + ":" + sr_pc["PC Checksheet Outstanding"].astype(str)
sr_pc = sr_pc.drop(columns=['PC Checksheet Outstanding'])
sr_pc = sr_pc.set_index('Sub System No')

# combine value as string 
sr_pc = sr_pc.assign(labels = sr_pc.groupby(level = 'Sub System No').cumcount()).groupby([sr_pc.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)

In [ ]:
sr_pc.head(3)

,Discipline,Discipline,Discipline,Discipline,Discipline
Sub System No,,,,,
068-01,C:1,NaN,NaN,NaN,NaN
068-08,C:1,NaN,NaN,NaN,NaN
068-09,C:2,NaN,NaN,NaN,NaN


In [ ]:
#adjustment jumlah column A/B/C/D
sr_pc.columns = ['PC Outstanding', 'A','B','C','D']
sr_pc["PC Outstanding"] = sr_pc["PC Outstanding"].astype(str) + "/" + sr_pc["A"].astype(str) + "/" + sr_pc["B"].astype(str) + "/" + sr_pc["C"].astype(str) + "/" + sr_pc["D"].astype(str)
sr_pc = sr_pc.drop(columns=['A','B','C','D'])

#sr_pc.columns = ['PC Outstanding', 'A','B','C']
#sr_pc["PC Outstanding"] = sr_pc["PC Outstanding"].astype(str) + "/" + sr_pc["A"].astype(str) + "/" + sr_pc["B"].astype(str) + "/" + sr_pc["C"].astype(str)
#sr_pc = sr_pc.drop(columns=['A','B','C'])

##4. Merge Certificate + SSO + Summary Report

In [ ]:
# merge cc_pc
cc_pc = pd.merge(sr_cc, sr_pc, left_on ="Sub System No", right_on="Sub System No" , how="outer")
# perlu ada fungsi untuk cleaning /nan/
cc_pc['CC Outstanding'] = cc_pc['CC Outstanding'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)
cc_pc['PC Outstanding'] = cc_pc['PC Outstanding'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)

In [ ]:
# Merge dengan data system owner sert_so dengan sr_pc
sert_sso = pd.merge(sso, cc_pc, left_on ="subsystem", right_on="Sub System No" , how="inner") # untuk menampilkan yg remaining saja
#sert_sso = pd.merge(sso, cc_pc, left_on ="subsystem", right_on="Sub System No" , how="left") # untuk semua list agar masuk

In [ ]:
sert_sso.head()

,subsystem,subsystem desc,system owner,CC Outstanding,PC Outstanding
0,068-01,Main Control Building,Yuniar,C:1,C:1
1,068-08,Substation at Sales Gas Metering,Andre Wisoyo,NaN,C:1
2,068-09,Guard House GPF,Yuniar,C:4,C:2
3,068-10,"Guard House, Security Post Gayam Public Road J...",Andre Wisoyo,C:1,C:1
4,068-11,Reception Building,Andre Wisoyo,C:3,C:1


#5. Connect to PunchList Data (cert+sso+summary report + punch)

In [ ]:
#pl = read_data(url_pl,'PL')

In [ ]:
# select only useful column
pl = pl[['Sub System No','Discipline','Category','Punchlist No','Punchlist Description','Status','Phase','Raised By']] 
pl = pl.loc[pl['Status']=='Outstanding'].drop(columns=['Status']) # hemat data yg outstanding saja yg di olah

#membuat buffer percabangan untuk di buat table dedicated dalam worksheet pl_A dan pl_B
pl_a = pl.loc[pl['Category'].isin(['A'])]
pl_b = pl.loc[pl['Category'].isin(['B'])]
pl_c = pl.loc[pl['Category'].isin(['C'])]

pl_ab = pl.loc[pl['Category'].isin(['A','B'])]   # saat include C, line ini akan di escape



In [ ]:
# Menghitung jumlah punch saja
pl_ab = pl_ab.groupby(['Sub System No','Discipline','Category'])['Punchlist No'].count().reset_index(name="jumlah")

# Combine text Discipline/Category/Jumlah C:(B=4)
pl_ab["Discipline"] = pl_ab["Discipline"].astype(str) + ":(" + pl_ab["Category"].astype(str) + "=" + pl_ab["jumlah"].astype(str) + ")"

pl_ab = pl_ab.drop(columns=['Category','jumlah'])
pl_ab = pl_ab.set_index('Sub System No')

# combine value as string ref
pl_ab = pl_ab.assign(labels = pl_ab.groupby(level = 'Sub System No').cumcount()).groupby([pl_ab.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)

In [ ]:
pl_ab.sample(4)

,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline
Sub System No,,,,,,,
520-03,L:(A=1),NaN,NaN,NaN,NaN,NaN,NaN
492-14,S:(B=2),NaN,NaN,NaN,NaN,NaN,NaN
063-21,C:(A=6),NaN,NaN,NaN,NaN,NaN,NaN
835-12,T:(A=2),NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#pl_ab.columns = ['Punch', 'A','B','C','D','E'] 
pl_ab.columns = ['Punch', 'A','B','C','D','E','F'] # jika ada error jumlah element
#pl_ab.columns = ['Punch', 'A','B','C','D','E','F','G','H','I']

#pl_ab["Punch"] = pl_ab["Punch"].astype(str) + "/" + pl_ab["A"].astype(str) + "/" + pl_ab["B"].astype(str) + "/" + pl_ab["C"].astype(str) + "/" + pl_ab["D"].astype(str) + "/" + pl_ab["E"].astype(str)
pl_ab["Punch"] = pl_ab["Punch"].astype(str) + "/" + pl_ab["A"].astype(str) + "/" + pl_ab["B"].astype(str) + "/" + pl_ab["C"].astype(str) + "/" + pl_ab["D"].astype(str) + "/" + pl_ab["E"].astype(str) + "/" + pl_ab["F"].astype(str)
#pl_ab["Punch"] = pl_ab["Punch"].astype(str) + "/" + pl_ab["A"].astype(str) + "/" + pl_ab["B"].astype(str) + "/" + pl_ab["C"].astype(str) + "/" + pl_ab["D"].astype(str) + "/" + pl_ab["E"].astype(str) + "/" + pl_ab["F"].astype(str) + "/" + pl_ab["G"].astype(str)

pl_ab = pl_ab.drop(columns=['A','B','C','D','E'])# jika ada error jumlah element
#pl_ab = pl_ab.drop(columns=['A','B','C','D','E','F'])# jika ada error jumlah element
#pl_ab = pl_ab.drop(columns=['A','B','C','D','E','F','G','H','I'])

pl_ab['Punch'] = pl_ab['Punch'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)


In [ ]:
# Merge dengan data subsystem owner sertificate sert_sso
sert_sso = pd.merge(sert_sso, pl_ab, left_on ="subsystem", right_on="Sub System No" , how="left")

In [ ]:
# save to gsheet diworksheet sc_workfront
#write_data(url_sc62, sert_sso, 'sc_workfront')


#6. Connect to System Priority untuk dimasukan dalam tabel prioritas (cert+sso+summary report + punch + tol)

In [ ]:
# Mengambil data dari SC62 worksheet priority dan clean up
priority = read_data(url_sc62,'priority')

# select only yg tol yes & column subsystem/subsystem desc/Tol
phase_2 = priority[['subsystem','subsystem desc','phase_2']]
phase_2 = phase_2.loc[phase_2['phase_2']=='yes']

# merge dengan table di sert_sso (worksheet sc_workfront)
sert_ssox = pd.merge(sert_sso, phase_2, left_on ="subsystem", right_on="subsystem" , how="left")
# reconfigure position column
sert_ssox = sert_ssox.reindex(columns=['subsystem','subsystem desc_x', 'Tol','system owner','CC Outstanding','PC Outstanding','Punch']) # letak posisi salah.

In [ ]:
phase_2.head(3)

,subsystem,subsystem desc,phase_2
309,241-09,H2S Recycle Compressor,yes
312,242-03,2nd Stage Membrane,yes
313,242-04,Permeate Recycle Compressor Suction Scrubber a...,yes


In [ ]:
'''
# membuat tabel sshoc tol
shoc_tol = df_tol[['Certificate No','System No','sshoc tol']]

#clean table dari blank row
nan_value = float("NaN")
shoc_tol.replace("", nan_value, inplace=True)
shoc_tol.dropna(subset = ["Certificate No"], inplace=True)

# select yg diperlukan gas in saja
shoc_tol = shoc_tol.loc[shoc_tol['sshoc tol']=='yes']

# merge SHOC tol dengan status certificate
shoc_cert = pd.merge(shoc_tol, sert_so, how="left").drop(columns=['Discipline']).sort_values(by=['Certificate No'])
'''

In [ ]:
# digabungkan dalam worksheet existing yg sudah ada isinya
#write_data(url_sc62, shoc_cert, 'shoc_sert_tol gasin')

#7. Connect to Data ITR

In [ ]:
#Ambil data ITR dan clean up
itr = itr[['Sub System No','Discipline','Tag No','ITR No','ITR Description','Status']]
# ambil hanya yang outstanding
itr = itr.loc[(itr['Status']=='Outstanding')]
# Membuat buffer percabangan untuk membuat tabel worksheet remaining ITR, utk digunakan nanti
itr_show = itr.drop(columns=['Status'])
# Menghitung jumlah ITR per subsystem
itr = itr.groupby(['Sub System No','Discipline'])['ITR No'].count().reset_index(name="jumlah")

# Combine text Discipline/Category/Jumlah C:1
itr["Discipline"] = itr["Discipline"].astype(str) + ":" + itr["jumlah"].astype(str)
itr = itr.drop(columns=['jumlah'])
itr = itr.set_index('Sub System No')

# combine value as string
itr = itr.assign(labels = itr.groupby(level = 'Sub System No').cumcount()).groupby([itr.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)
itr.head(3)

,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline
Sub System No,,,,,,,,,
009-06,L:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
062-04,C:6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
063-01,C:4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
itr.columns = ['ITR','A','B','C','D','E','F','G','H']

itr["ITR"] = itr["ITR"].astype(str) + "/" + itr["A"].astype(str) + "/" + itr["B"].astype(str) + "/" + itr["C"].astype(str) + "/" + itr["D"].astype(str) + "/" + itr["E"].astype(str) + "/" + itr["F"].astype(str) + "/" + itr["G"].astype(str) + "/" + itr["H"].astype(str)
itr = itr.drop(columns=['A','B','C','D','E','F','G','H'])

# cleaning /nan/
itr['ITR'] = itr['ITR'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)

In [ ]:
# merge dengan table di sert_sso (worksheet sc_workfront)
sert_ssox2 = pd.merge(sert_ssox, itr, left_on ="subsystem", right_on="Sub System No" , how="left")
sert_ssox2 = sert_ssox2.reindex(columns=['subsystem','subsystem desc_x', 'Tol','system owner', 'ITR', 'CC Outstanding','PC Outstanding','Punch'])

In [ ]:
# save to gsheet diworksheet sc_workfront
write_data(url_sc62, sert_ssox2, 'sc_workfront')

#8. Refresh Outstanding ITR/PC/CC/ITR

In [ ]:
# merge dengan table phase_2 
itr_show = pd.merge(itr_show, phase_2 , left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur phase_2 
itr_show = itr_show.loc[(itr_show['phase_2']=='yes')]
itr_show = itr_show.drop(columns=["subsystem"])

# Update worksheet remaining ITR 
write_data(url_sc62, itr_show, 'itr')

In [ ]:
#Ambil data CC dan clean up
cc = cc[['Sub System No','Discipline','Tag No','Checksheet Description','Status']]

# ambil hanya yang outstanding
cc = cc.loc[(cc['Status']=='Outstanding')]
cc = cc.drop(columns=['Status'])

# merge dengan table phase_2
cc = pd.merge(cc, phase_2, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur phase_2
cc = cc.loc[(cc['phase_2']=='yes')]
cc = cc.drop(columns=["subsystem"])

# Update worksheet remaining CC
write_data(url_sc62, cc, 'cc')

In [ ]:
# merge dengan table phase_2
pl_a = pd.merge(pl_a, phase_2, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur phase_2
pl_a = pl_a.loc[(pl_a['phase_2']=='yes')]
pl_a = pl_a.drop(columns=["subsystem"])

# Update worksheet remaining PL-A
write_data(url_sc62, pl_a, 'pl_A')

In [ ]:
#Ambil data PC dan clean up
pc = pc[['Sub System No','Discipline','Tag No','Checksheet No','Checksheet Description','Status']]

# ambil hanya yang outstanding
pc = pc.loc[(pc['Status']=='Outstanding')]
pc = pc.drop(columns=['Status'])

# merge dengan table phase_2
pc = pd.merge(pc, phase_2, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur phase_2
pc = pc.loc[(pc['phase_2']=='yes')]
pc = pc.drop(columns=["subsystem"])

# Update worksheet remaining PC # di pindahkan ke bawah
write_data(url_sc62, pc, 'pc')

In [ ]:
# merge dengan table phase_2
pl_b = pd.merge(pl_b, phase_2, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur phase_2
pl_b = pl_b.loc[(pl_b['phase_2']=='yes')]
pl_b = pl_b.drop(columns=["subsystem"])

# Update worksheet remaining PL-B
write_data(url_sc62, pl_b, 'pl_B')

#9. Connect to Transmital SC17 (cc dan pc)

In [ ]:
# terhubung dengan SC17
url_sc17 = 'https://docs.google.com/spreadsheets/d/1nrdtZOiToC_kW95a8rKR_-J8hH5Io5Rp_JG2a8S2T3o/edit#gid=1274212197'

sc17_cc = read_data(url_sc17,'CS Monitoring ')
sc17_pc = read_data(url_sc17,'PC Monitoring')
sc17_cert = read_data(url_sc17,'Certificate Monitoring')

In [ ]:
#ambil data transmital PC yg diperlukan dan clean up
sc17_cc = sc17_cc[['Subsystem','Date','   Discipline','Tag No','Checksheet No','Status','Remarks','Transmital No']]
#merge data
cc_x1 = pd.merge(cc, sc17_cc, left_on ="Tag No", right_on="Tag No" , how="left")

#di atur kolomnya
cc_x1 = cc_x1.reindex(columns=['Sub System No','Discipline', 'Tag No','Checksheet Description', 'subsystem desc','Tol','Date','Status','Remarks','Transmital No'])
cc_x1 = cc_x1.rename(columns={"Date": "SC17_submit Date", "Remarks": "SC17 Status"})

# write cc
#write_data(url_sc62, cc_x1, 'cc')

In [ ]:
#ambil data transmital PC yg diperlukan dan clean up
sc17_pc = sc17_pc[['Subsystem No.','Date','Discipline','Tag No.','Checksheet No','Checksheet Description','Status','Remarks','Transmital No']]

#merge data
#pc_x1 = pd.merge(pc, sc17_pc, left_on ="Tag No", right_on="Tag No." , how="left") # ada merging yg kurang pas, karena tag number tidak unique di SC17
pc_x1 = pd.merge(pc, sc17_pc, left_on =["Tag No","Checksheet No"], right_on=["Tag No.","Checksheet No"] , how="left") # absorbing comment/finding bug p Adang, perlu check 2 column


#di atur kolomnya
pc_x1 = pc_x1.reindex(columns=['Sub System No','Discipline_x', 'Tag No','Checksheet No', 'Checksheet Description_x',
'Tol','Date','Remarks','Transmital No'])
pc_x1 = pc_x1.rename(columns={"Date": "SC17_submit Date", "Remarks": "SC17 Status"})

# write pc
write_data(url_sc62, pc_x1, 'pc')

In [ ]:
sc17_cert = read_data(url_sc17,'Certificate Monitoring')

In [ ]:
#ambil data transmital cert yg diperlukan dan clean up
#sc17_cert = sc17_cert[['System/sub-system','Date','Discipline','Certificate','Re-submit','Remarks','Transmital No']]
sc17_cert = sc17_cert[['Sub-System No.','Date','Discipline','Certificate','Submit','Remarks','Transmital No']]